In [13]:
from functools import partial
import numpy as np
import tensorflow as tf

In [7]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")

Extracting MNIST-data\train-images-idx3-ubyte.gz
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz


# 8a

In [8]:
tf.reset_default_graph()

n_inputs = 28 * 28
n_hidden = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

he_init = tf.keras.initializers.he_normal()
my_dense_layer = partial(tf.layers.dense, activation=tf.nn.elu, kernel_initializer=he_init)

with tf.name_scope("dnn"):
    hidden1 = my_dense_layer(X, n_hidden, name="hidden1")
    hidden2 = my_dense_layer(hidden1, n_hidden, name="hidden2")
    hidden3 = my_dense_layer(hidden2, n_hidden, name="hidden3")
    hidden4 = my_dense_layer(hidden3, n_hidden, name="hidden4")
    hidden5 = my_dense_layer(hidden4, n_hidden, name="hidden5")
    logits = my_dense_layer(hidden5, n_hidden, name="outputs")
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
with tf.name_scope("train"):
    optimizer = tf.train.MomentumOptimizer(learning_rate=0.01, momentum=0.9)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [9]:
n_epochs = 20
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for _ in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
        print(epoch, "Test accuracy:", accuracy_val)
        
    saver.save(sess, "./8a.ckpt")

0 Test accuracy: 0.951
1 Test accuracy: 0.9662
2 Test accuracy: 0.9653
3 Test accuracy: 0.9714
4 Test accuracy: 0.9719
5 Test accuracy: 0.9688
6 Test accuracy: 0.9691
7 Test accuracy: 0.9699
8 Test accuracy: 0.9769
9 Test accuracy: 0.9747
10 Test accuracy: 0.9783
11 Test accuracy: 0.9768
12 Test accuracy: 0.9779
13 Test accuracy: 0.9791
14 Test accuracy: 0.9785
15 Test accuracy: 0.9766
16 Test accuracy: 0.9792
17 Test accuracy: 0.9781
18 Test accuracy: 0.9776
19 Test accuracy: 0.9763


## 8b

In [10]:
tf.reset_default_graph()

n_inputs = 28 * 28
n_hidden = 100
n_outputs = 5

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

he_init = tf.keras.initializers.he_normal()
my_dense_layer = partial(tf.layers.dense, activation=tf.nn.elu, kernel_initializer=he_init)

with tf.name_scope("dnn"):
    hidden1 = my_dense_layer(X, n_hidden, name="hidden1")
    hidden2 = my_dense_layer(hidden1, n_hidden, name="hidden2")
    hidden3 = my_dense_layer(hidden2, n_hidden, name="hidden3")
    hidden4 = my_dense_layer(hidden3, n_hidden, name="hidden4")
    hidden5 = my_dense_layer(hidden4, n_hidden, name="hidden5")
    logits = my_dense_layer(hidden5, n_hidden, activation=tf.nn.softmax, name="outputs")
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [14]:
train_indices = mnist.train.labels < 5
train_X = mnist.train.images[train_indices]
train_y = mnist.train.labels[train_indices]
validation_indices = mnist.validation.labels < 5
validation_X = mnist.validation.images[validation_indices]
validation_y = mnist.validation.labels[validation_indices]
test_indices = mnist.test.labels < 5
test_X = mnist.test.images[test_indices]
test_y = mnist.test.labels[test_indices]

In [20]:
n_epochs = 100
batch_size = 50
best_loss = 1.0
steps = 0
max_steps = 20

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        randomized_indices = np.random.permutation(len(train_X))
        for batch_indices in np.array_split(randomized_indices, len(train_X) // batch_size):
            X_batch, y_batch = train_X[batch_indices], train_y[batch_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, accuracy_val = sess.run([loss, accuracy], feed_dict={X: validation_X, y: validation_y})
        if loss_val < best_loss:
            best_loss = loss_val
            saver.save(sess, "./8b_checkpoint.ckpt")
            steps = 0
        else:
            steps += 1
            if steps >= max_steps:
                print("Stopping early")
                break
        print(epoch, "Current loss:", loss_val, "Best loss:", best_loss, "Validation accuracy:", accuracy_val)
        
with tf.Session() as sess:
    saver.restore(sess, "./8b_checkpoint.ckpt")
    accuracy_val = accuracy.eval(feed_dict={X: test_X, y: test_y})
    print("Test accuracy:", accuracy_val)
    saver.save(sess, "./8b.ckpt")

0 Current loss: 0.1099464 Best loss: 0.1099464 Validation accuracy: 0.9714621
1 Current loss: 0.10241289 Best loss: 0.10241289 Validation accuracy: 0.97263485
2 Current loss: 0.2703067 Best loss: 0.10241289 Validation accuracy: 0.9640344
3 Current loss: 0.05638243 Best loss: 0.05638243 Validation accuracy: 0.98397183
4 Current loss: 0.07955036 Best loss: 0.05638243 Validation accuracy: 0.9820172
5 Current loss: 0.08936607 Best loss: 0.05638243 Validation accuracy: 0.9816263
6 Current loss: 0.12252025 Best loss: 0.05638243 Validation accuracy: 0.97615325
7 Current loss: 0.09525351 Best loss: 0.05638243 Validation accuracy: 0.98006254
8 Current loss: 0.08588001 Best loss: 0.05638243 Validation accuracy: 0.9824081
9 Current loss: 0.084384054 Best loss: 0.05638243 Validation accuracy: 0.9847537
10 Current loss: 0.073684186 Best loss: 0.05638243 Validation accuracy: 0.9804535
11 Current loss: 0.07109239 Best loss: 0.05638243 Validation accuracy: 0.9859265
12 Current loss: 0.08198538 Best lo

## 8c

In [21]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.exceptions import NotFittedError

class DNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, n_hidden_layers=5, n_neurons=100, optimizer_class=tf.train.AdamOptimizer,
                 learning_rate=0.01, batch_size=20, activation=tf.nn.elu, initializer=he_init,
                 batch_norm_momentum=None, dropout_rate=None, random_state=None):
        """Initialize the DNNClassifier by simply storing all the hyperparameters."""
        self.n_hidden_layers = n_hidden_layers
        self.n_neurons = n_neurons
        self.optimizer_class = optimizer_class
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.activation = activation
        self.initializer = initializer
        self.batch_norm_momentum = batch_norm_momentum
        self.dropout_rate = dropout_rate
        self.random_state = random_state
        self._session = None

    def _dnn(self, inputs):
        """Build the hidden layers, with support for batch normalization and dropout."""
        for layer in range(self.n_hidden_layers):
            if self.dropout_rate:
                inputs = tf.layers.dropout(inputs, self.dropout_rate, training=self._training)
            inputs = tf.layers.dense(inputs, self.n_neurons,
                                     kernel_initializer=self.initializer,
                                     name="hidden%d" % (layer + 1))
            if self.batch_norm_momentum:
                inputs = tf.layers.batch_normalization(inputs, momentum=self.batch_norm_momentum,
                                                       training=self._training)
            inputs = self.activation(inputs, name="hidden%d_out" % (layer + 1))
        return inputs

    def _build_graph(self, n_inputs, n_outputs):
        """Build the same model as earlier"""
        if self.random_state is not None:
            tf.set_random_seed(self.random_state)
            np.random.seed(self.random_state)

        X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
        y = tf.placeholder(tf.int32, shape=(None), name="y")

        if self.batch_norm_momentum or self.dropout_rate:
            self._training = tf.placeholder_with_default(False, shape=(), name='training')
        else:
            self._training = None

        dnn_outputs = self._dnn(X)

        logits = tf.layers.dense(dnn_outputs, n_outputs, kernel_initializer=he_init, name="logits")
        Y_proba = tf.nn.softmax(logits, name="Y_proba")

        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                                  logits=logits)
        loss = tf.reduce_mean(xentropy, name="loss")

        optimizer = self.optimizer_class(learning_rate=self.learning_rate)
        training_op = optimizer.minimize(loss)

        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

        init = tf.global_variables_initializer()
        saver = tf.train.Saver()

        # Make the important operations available easily through instance variables
        self._X, self._y = X, y
        self._Y_proba, self._loss = Y_proba, loss
        self._training_op, self._accuracy = training_op, accuracy
        self._init, self._saver = init, saver

    def close_session(self):
        if self._session:
            self._session.close()

    def _get_model_params(self):
        """Get all variable values (used for early stopping, faster than saving to disk)"""
        with self._graph.as_default():
            gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
        return {gvar.op.name: value for gvar, value in zip(gvars, self._session.run(gvars))}

    def _restore_model_params(self, model_params):
        """Set all variables to the given values (for early stopping, faster than loading from disk)"""
        gvar_names = list(model_params.keys())
        assign_ops = {gvar_name: self._graph.get_operation_by_name(gvar_name + "/Assign")
                      for gvar_name in gvar_names}
        init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
        feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
        self._session.run(assign_ops, feed_dict=feed_dict)

    def fit(self, X, y, n_epochs=100, X_valid=None, y_valid=None):
        """Fit the model to the training set. If X_valid and y_valid are provided, use early stopping."""
        self.close_session()

        # infer n_inputs and n_outputs from the training set.
        n_inputs = X.shape[1]
        self.classes_ = np.unique(y)
        n_outputs = len(self.classes_)
        
        # Translate the labels vector to a vector of sorted class indices, containing
        # integers from 0 to n_outputs - 1.
        # For example, if y is equal to [8, 8, 9, 5, 7, 6, 6, 6], then the sorted class
        # labels (self.classes_) will be equal to [5, 6, 7, 8, 9], and the labels vector
        # will be translated to [3, 3, 4, 0, 2, 1, 1, 1]
        self.class_to_index_ = {label: index
                                for index, label in enumerate(self.classes_)}
        y = np.array([self.class_to_index_[label]
                      for label in y], dtype=np.int32)
        
        self._graph = tf.Graph()
        with self._graph.as_default():
            self._build_graph(n_inputs, n_outputs)
            # extra ops for batch normalization
            extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

        # needed in case of early stopping
        max_checks_without_progress = 20
        checks_without_progress = 0
        best_loss = np.infty
        best_params = None
        
        # Now train the model!
        self._session = tf.Session(graph=self._graph)
        with self._session.as_default() as sess:
            self._init.run()
            for epoch in range(n_epochs):
                rnd_idx = np.random.permutation(len(X))
                for rnd_indices in np.array_split(rnd_idx, len(X) // self.batch_size):
                    X_batch, y_batch = X[rnd_indices], y[rnd_indices]
                    feed_dict = {self._X: X_batch, self._y: y_batch}
                    if self._training is not None:
                        feed_dict[self._training] = True
                    sess.run(self._training_op, feed_dict=feed_dict)
                    if extra_update_ops:
                        sess.run(extra_update_ops, feed_dict=feed_dict)
                if X_valid is not None and y_valid is not None:
                    loss_val, acc_val = sess.run([self._loss, self._accuracy],
                                                 feed_dict={self._X: X_valid,
                                                            self._y: y_valid})
                    if loss_val < best_loss:
                        best_params = self._get_model_params()
                        best_loss = loss_val
                        checks_without_progress = 0
                    else:
                        checks_without_progress += 1
                    print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
                        epoch, loss_val, best_loss, acc_val * 100))
                    if checks_without_progress > max_checks_without_progress:
                        print("Early stopping!")
                        break
                else:
                    loss_train, acc_train = sess.run([self._loss, self._accuracy],
                                                     feed_dict={self._X: X_batch,
                                                                self._y: y_batch})
                    print("{}\tLast training batch loss: {:.6f}\tAccuracy: {:.2f}%".format(
                        epoch, loss_train, acc_train * 100))
            # If we used early stopping then rollback to the best model found
            if best_params:
                self._restore_model_params(best_params)
            return self

    def predict_proba(self, X):
        if not self._session:
            raise NotFittedError("This %s instance is not fitted yet" % self.__class__.__name__)
        with self._session.as_default() as sess:
            return self._Y_proba.eval(feed_dict={self._X: X})

    def predict(self, X):
        class_indices = np.argmax(self.predict_proba(X), axis=1)
        return np.array([[self.classes_[class_index]]
                         for class_index in class_indices], np.int32)

    def save(self, path):
        self._saver.save(self._session, path)

In [22]:
dnn_clf = DNNClassifier(random_state=42)
dnn_clf.fit(train_X, train_y, n_epochs=1000, X_valid=validation_X, y_valid=validation_y)

0	Validation loss: 1.174311	Best loss: 1.174311	Accuracy: 76.43%
1	Validation loss: 0.142328	Best loss: 0.142328	Accuracy: 97.58%
2	Validation loss: 0.116103	Best loss: 0.116103	Accuracy: 97.62%
3	Validation loss: 0.362416	Best loss: 0.116103	Accuracy: 95.07%
4	Validation loss: 0.193965	Best loss: 0.116103	Accuracy: 97.77%
5	Validation loss: 1.160837	Best loss: 0.116103	Accuracy: 96.91%
6	Validation loss: 0.178270	Best loss: 0.116103	Accuracy: 97.03%
7	Validation loss: 0.214997	Best loss: 0.116103	Accuracy: 97.81%
8	Validation loss: 0.280040	Best loss: 0.116103	Accuracy: 97.69%
9	Validation loss: 0.705137	Best loss: 0.116103	Accuracy: 70.64%
10	Validation loss: 0.408118	Best loss: 0.116103	Accuracy: 79.24%
11	Validation loss: 0.602409	Best loss: 0.116103	Accuracy: 79.28%
12	Validation loss: 0.556710	Best loss: 0.116103	Accuracy: 79.79%
13	Validation loss: 0.585478	Best loss: 0.116103	Accuracy: 78.93%
14	Validation loss: 1.277399	Best loss: 0.116103	Accuracy: 56.45%
15	Validation loss: 

DNNClassifier(activation=<function elu at 0x000000DF7F9CD0D0>,
       batch_norm_momentum=None, batch_size=20, dropout_rate=None,
       initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x000000DF13926A20>,
       learning_rate=0.01, n_hidden_layers=5, n_neurons=100,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42)

In [23]:
from sklearn.metrics import accuracy_score

y_pred = dnn_clf.predict(test_X)
accuracy_score(test_y, y_pred)

0.9785950574041642

In [25]:
from sklearn.model_selection import RandomizedSearchCV

def leaky_relu(alpha=0.01):
    def parameterized_leaky_relu(z, name=None):
        return tf.maximum(alpha * z, z, name=name)
    return parameterized_leaky_relu

param_distribs = {
    "n_neurons": [10, 30, 50, 70, 90, 100, 120, 140, 160],
    "batch_size": [10, 50, 100, 500],
    "learning_rate": [0.01, 0.02, 0.05, 0.1],
    "activation": [tf.nn.relu, tf.nn.elu, leaky_relu(alpha=0.01), leaky_relu(alpha=0.1)],
    "n_hidden_layers": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "optimizer_class": [tf.train.AdamOptimizer, partial(tf.train.MomentumOptimizer, momentum=0.95)],
}

rnd_search = RandomizedSearchCV(DNNClassifier(random_state=42), param_distribs, n_iter=50, random_state=42, verbose=2)
fit_params={"X_valid": validation_X, "y_valid": validation_y, "n_epochs": 1000}
rnd_search.fit(train_X, train_y, **fit_params)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>, n_neurons=160, n_hidden_layers=7, learning_rate=0.01, batch_size=50, activation=<function leaky_relu.<locals>.parameterized_leaky_relu at 0x000000DF17124E18> 
0	Validation loss: 0.396365	Best loss: 0.396365	Accuracy: 86.20%
1	Validation loss: 0.316186	Best loss: 0.316186	Accuracy: 90.97%
2	Validation loss: 0.181531	Best loss: 0.181531	Accuracy: 95.27%
3	Validation loss: 0.238905	Best loss: 0.181531	Accuracy: 92.77%
4	Validation loss: 0.146108	Best loss: 0.146108	Accuracy: 96.21%
5	Validation loss: 0.129314	Best loss: 0.129314	Accuracy: 97.11%
6	Validation loss: 0.121177	Best loss: 0.121177	Accuracy: 96.95%
7	Validation loss: 0.090877	Best loss: 0.090877	Accuracy: 97.34%
8	Validation loss: 0.096111	Best loss: 0.090877	Accuracy: 97.38%
9	Validation loss: 0.105575	Best loss: 0.090877	Accuracy: 97.22%
10	Validation loss: 0.088909	Best loss: 0.088909	Ac

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   36.9s remaining:    0.0s


0	Validation loss: 0.135305	Best loss: 0.135305	Accuracy: 96.72%
1	Validation loss: 0.130770	Best loss: 0.130770	Accuracy: 96.52%
2	Validation loss: 0.089658	Best loss: 0.089658	Accuracy: 97.38%
3	Validation loss: 0.118707	Best loss: 0.089658	Accuracy: 97.77%
4	Validation loss: 0.093973	Best loss: 0.089658	Accuracy: 97.34%
5	Validation loss: 0.108280	Best loss: 0.089658	Accuracy: 97.19%
6	Validation loss: 0.097972	Best loss: 0.089658	Accuracy: 97.85%
7	Validation loss: 20.817612	Best loss: 0.089658	Accuracy: 63.17%
8	Validation loss: 2.050639	Best loss: 0.089658	Accuracy: 75.96%
9	Validation loss: 2.108161	Best loss: 0.089658	Accuracy: 75.22%
10	Validation loss: 2.177601	Best loss: 0.089658	Accuracy: 71.19%
11	Validation loss: 1.922042	Best loss: 0.089658	Accuracy: 68.80%
12	Validation loss: 0.935331	Best loss: 0.089658	Accuracy: 86.94%
13	Validation loss: 0.659626	Best loss: 0.089658	Accuracy: 89.41%
14	Validation loss: 0.632097	Best loss: 0.089658	Accuracy: 90.23%
15	Validation loss:

28	Validation loss: 0.129856	Best loss: 0.043240	Accuracy: 98.71%
Early stopping!
[CV]  optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>, n_neurons=140, n_hidden_layers=3, learning_rate=0.01, batch_size=50, activation=<function relu at 0x000000DF7F9EA400>, total=  20.7s
[CV] optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>, n_neurons=140, n_hidden_layers=3, learning_rate=0.01, batch_size=50, activation=<function relu at 0x000000DF7F9EA400> 
0	Validation loss: 0.083975	Best loss: 0.083975	Accuracy: 97.65%
1	Validation loss: 0.072628	Best loss: 0.072628	Accuracy: 97.50%
2	Validation loss: 0.109453	Best loss: 0.072628	Accuracy: 97.50%
3	Validation loss: 0.093471	Best loss: 0.072628	Accuracy: 98.44%
4	Validation loss: 0.051594	Best loss: 0.051594	Accuracy: 98.51%
5	Validation loss: 0.072373	Best loss: 0.051594	Accuracy: 98.44%
6	Validation loss: 0.063799	Best loss: 0.051594	Accuracy: 98.59%
7	Validation loss: 0.068632	Best loss: 0.051594	Accura

0	Validation loss: 0.072134	Best loss: 0.072134	Accuracy: 97.89%
1	Validation loss: 0.064736	Best loss: 0.064736	Accuracy: 97.97%
2	Validation loss: 0.044514	Best loss: 0.044514	Accuracy: 98.59%
3	Validation loss: 0.050612	Best loss: 0.044514	Accuracy: 98.48%
4	Validation loss: 0.046069	Best loss: 0.044514	Accuracy: 98.67%
5	Validation loss: 0.056978	Best loss: 0.044514	Accuracy: 98.40%
6	Validation loss: 0.053524	Best loss: 0.044514	Accuracy: 98.79%
7	Validation loss: 0.053213	Best loss: 0.044514	Accuracy: 98.59%
8	Validation loss: 0.053391	Best loss: 0.044514	Accuracy: 98.71%
9	Validation loss: 0.048457	Best loss: 0.044514	Accuracy: 98.83%
10	Validation loss: 0.052939	Best loss: 0.044514	Accuracy: 98.83%
11	Validation loss: 0.046530	Best loss: 0.044514	Accuracy: 99.02%
12	Validation loss: 0.044660	Best loss: 0.044514	Accuracy: 99.06%
13	Validation loss: 0.047862	Best loss: 0.044514	Accuracy: 99.02%
14	Validation loss: 0.046891	Best loss: 0.044514	Accuracy: 99.02%
15	Validation loss: 

...

16	Validation loss: 0.114585	Best loss: 0.098790	Accuracy: 97.03%
17	Validation loss: 0.117959	Best loss: 0.098790	Accuracy: 97.19%
18	Validation loss: 0.123770	Best loss: 0.098790	Accuracy: 97.03%
19	Validation loss: 0.126820	Best loss: 0.098790	Accuracy: 97.07%
20	Validation loss: 0.133012	Best loss: 0.098790	Accuracy: 97.38%
21	Validation loss: 0.129494	Best loss: 0.098790	Accuracy: 96.99%
22	Validation loss: 0.115236	Best loss: 0.098790	Accuracy: 97.26%
23	Validation loss: 0.111224	Best loss: 0.098790	Accuracy: 96.99%
24	Validation loss: 0.131891	Best loss: 0.098790	Accuracy: 97.30%
25	Validation loss: 0.120716	Best loss: 0.098790	Accuracy: 97.69%
26	Validation loss: 0.134214	Best loss: 0.098790	Accuracy: 96.76%
27	Validation loss: 0.132455	Best loss: 0.098790	Accuracy: 96.99%
28	Validation loss: 0.113042	Best loss: 0.098790	Accuracy: 97.65%
29	Validation loss: 0.126996	Best loss: 0.098790	Accuracy: 97.50%
Early stopping!
[CV]  optimizer_class=<class 'tensorflow.python.training.ada

7	Validation loss: 7515095433216.000000	Best loss: 1733146.250000	Accuracy: 87.57%
8	Validation loss: 6141585653760.000000	Best loss: 1733146.250000	Accuracy: 90.11%
9	Validation loss: 5626194821120.000000	Best loss: 1733146.250000	Accuracy: 87.84%
10	Validation loss: 3694580989952.000000	Best loss: 1733146.250000	Accuracy: 90.62%
11	Validation loss: 5249920663552.000000	Best loss: 1733146.250000	Accuracy: 85.03%
12	Validation loss: 2027040669696.000000	Best loss: 1733146.250000	Accuracy: 92.61%
13	Validation loss: 1655759568896.000000	Best loss: 1733146.250000	Accuracy: 91.01%
14	Validation loss: 1946210009088.000000	Best loss: 1733146.250000	Accuracy: 89.95%
15	Validation loss: 2651242364928.000000	Best loss: 1733146.250000	Accuracy: 88.78%
16	Validation loss: 1089423802368.000000	Best loss: 1733146.250000	Accuracy: 93.78%
17	Validation loss: 1155723165696.000000	Best loss: 1733146.250000	Accuracy: 93.12%
18	Validation loss: 1073798905856.000000	Best loss: 1733146.250000	Accuracy: 94

[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 66.8min finished


0	Validation loss: 0.051609	Best loss: 0.051609	Accuracy: 98.48%
1	Validation loss: 0.040726	Best loss: 0.040726	Accuracy: 98.51%
2	Validation loss: 0.061499	Best loss: 0.040726	Accuracy: 98.12%
3	Validation loss: 0.037207	Best loss: 0.037207	Accuracy: 98.87%
4	Validation loss: 0.033410	Best loss: 0.033410	Accuracy: 99.06%
5	Validation loss: 0.025029	Best loss: 0.025029	Accuracy: 99.26%
6	Validation loss: 0.039156	Best loss: 0.025029	Accuracy: 98.94%
7	Validation loss: 0.032627	Best loss: 0.025029	Accuracy: 99.02%
8	Validation loss: 0.040720	Best loss: 0.025029	Accuracy: 98.79%
9	Validation loss: 0.032792	Best loss: 0.025029	Accuracy: 98.98%
10	Validation loss: 0.049675	Best loss: 0.025029	Accuracy: 98.91%
11	Validation loss: 0.019880	Best loss: 0.019880	Accuracy: 99.26%
12	Validation loss: 0.027693	Best loss: 0.019880	Accuracy: 99.34%
13	Validation loss: 0.022122	Best loss: 0.019880	Accuracy: 99.53%
14	Validation loss: 0.020762	Best loss: 0.019880	Accuracy: 99.45%
15	Validation loss: 

RandomizedSearchCV(cv=None, error_score='raise',
          estimator=DNNClassifier(activation=<function elu at 0x000000DF7F9CD0D0>,
       batch_norm_momentum=None, batch_size=20, dropout_rate=None,
       initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x000000DF13926A20>,
       learning_rate=0.01, n_hidden_layers=5, n_neurons=100,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42),
          fit_params=None, iid=True, n_iter=50, n_jobs=1,
          param_distributions={'n_neurons': [10, 30, 50, 70, 90, 100, 120, 140, 160], 'batch_size': [10, 50, 100, 500], 'learning_rate': [0.01, 0.02, 0.05, 0.1], 'activation': [<function relu at 0x000000DF7F9EA400>, <function elu at 0x000000DF7F9CD0D0>, <function leaky_relu.<locals>.parameterized_leaky_relu a... functools.partial(<class 'tensorflow.python.training.momentum.MomentumOptimizer'>, momentum=0.95)]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,


In [26]:
rnd_search.best_params_

{'activation': <function __main__.leaky_relu.<locals>.parameterized_leaky_relu>,
 'batch_size': 50,
 'learning_rate': 0.02,
 'n_hidden_layers': 4,
 'n_neurons': 160,
 'optimizer_class': functools.partial(<class 'tensorflow.python.training.momentum.MomentumOptimizer'>, momentum=0.95)}

In [27]:
y_pred = rnd_search.predict(test_X)
accuracy_score(test_y, y_pred)

0.9929947460595446

In [28]:
rnd_search.best_estimator_.save("./8c.ckpt")

## 8d

In [35]:
dnn_clf = DNNClassifier(n_hidden_layers=4, n_neurons=160, optimizer_class=partial(tf.train.MomentumOptimizer, momentum=0.95),
                 learning_rate=0.02, batch_size=50, activation=leaky_relu(alpha=0.01), random_state=42)
dnn_clf.fit(train_X, train_y, n_epochs=1000, X_valid=validation_X, y_valid=validation_y)

0	Validation loss: 0.053766	Best loss: 0.053766	Accuracy: 98.44%
1	Validation loss: 0.047046	Best loss: 0.047046	Accuracy: 98.20%
2	Validation loss: 0.089815	Best loss: 0.047046	Accuracy: 97.46%
3	Validation loss: 0.032686	Best loss: 0.032686	Accuracy: 99.06%
4	Validation loss: 0.026496	Best loss: 0.026496	Accuracy: 99.22%
5	Validation loss: 0.039467	Best loss: 0.026496	Accuracy: 99.18%
6	Validation loss: 0.025174	Best loss: 0.025174	Accuracy: 99.22%
7	Validation loss: 0.040610	Best loss: 0.025174	Accuracy: 99.10%
8	Validation loss: 0.023979	Best loss: 0.023979	Accuracy: 99.26%
9	Validation loss: 0.026132	Best loss: 0.023979	Accuracy: 99.34%
10	Validation loss: 0.021548	Best loss: 0.021548	Accuracy: 99.61%
11	Validation loss: 0.027131	Best loss: 0.021548	Accuracy: 99.22%
12	Validation loss: 0.027913	Best loss: 0.021548	Accuracy: 99.30%
13	Validation loss: 0.023908	Best loss: 0.021548	Accuracy: 99.37%
14	Validation loss: 0.020749	Best loss: 0.020749	Accuracy: 99.57%
15	Validation loss: 

DNNClassifier(activation=<function leaky_relu.<locals>.parameterized_leaky_relu at 0x000000DF1DE42D08>,
       batch_norm_momentum=None, batch_size=50, dropout_rate=None,
       initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x000000DF13926A20>,
       learning_rate=0.02, n_hidden_layers=4, n_neurons=160,
       optimizer_class=functools.partial(<class 'tensorflow.python.training.momentum.MomentumOptimizer'>, momentum=0.95),
       random_state=42)

In [36]:
y_pred = dnn_clf.predict(test_X)
accuracy_score(test_y, y_pred)

0.9957190114808329

In [37]:
dnn_clf_bn = DNNClassifier(n_hidden_layers=4, n_neurons=160, optimizer_class=partial(tf.train.MomentumOptimizer, momentum=0.95),
                           learning_rate=0.02, batch_size=50, activation=leaky_relu(alpha=0.01), random_state=42,
                           batch_norm_momentum=0.95)
dnn_clf_bn.fit(train_X, train_y, n_epochs=1000, X_valid=validation_X, y_valid=validation_y)

0	Validation loss: 0.038765	Best loss: 0.038765	Accuracy: 98.75%
1	Validation loss: 0.028899	Best loss: 0.028899	Accuracy: 99.10%
2	Validation loss: 0.039919	Best loss: 0.028899	Accuracy: 98.83%
3	Validation loss: 0.038799	Best loss: 0.028899	Accuracy: 98.87%
4	Validation loss: 0.023390	Best loss: 0.023390	Accuracy: 99.37%
5	Validation loss: 0.027679	Best loss: 0.023390	Accuracy: 99.30%
6	Validation loss: 0.024720	Best loss: 0.023390	Accuracy: 99.45%
7	Validation loss: 0.027166	Best loss: 0.023390	Accuracy: 99.14%
8	Validation loss: 0.023431	Best loss: 0.023390	Accuracy: 99.41%
9	Validation loss: 0.029456	Best loss: 0.023390	Accuracy: 99.37%
10	Validation loss: 0.017717	Best loss: 0.017717	Accuracy: 99.73%
11	Validation loss: 0.024462	Best loss: 0.017717	Accuracy: 99.34%
12	Validation loss: 0.022002	Best loss: 0.017717	Accuracy: 99.49%
13	Validation loss: 0.020526	Best loss: 0.017717	Accuracy: 99.65%
14	Validation loss: 0.022210	Best loss: 0.017717	Accuracy: 99.53%
15	Validation loss: 

DNNClassifier(activation=<function leaky_relu.<locals>.parameterized_leaky_relu at 0x000000DF1DE42D90>,
       batch_norm_momentum=0.95, batch_size=50, dropout_rate=None,
       initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x000000DF13926A20>,
       learning_rate=0.02, n_hidden_layers=4, n_neurons=160,
       optimizer_class=functools.partial(<class 'tensorflow.python.training.momentum.MomentumOptimizer'>, momentum=0.95),
       random_state=42)

In [38]:
y_pred = dnn_clf_bn.predict(test_X)
accuracy_score(test_y, y_pred)

0.9949406499318934

In [39]:
param_distribs = {
    "n_neurons": [10, 30, 50, 70, 90, 100, 120, 140, 160],
    "batch_size": [10, 50, 100, 500],
    "learning_rate": [0.01, 0.02, 0.05, 0.1],
    "activation": [tf.nn.relu, tf.nn.elu, leaky_relu(alpha=0.01), leaky_relu(alpha=0.1)],
    "n_hidden_layers": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "optimizer_class": [tf.train.AdamOptimizer, partial(tf.train.MomentumOptimizer, momentum=0.95)],
    "batch_norm_momentum": [0.9, 0.95, 0.98, 0.99, 0.999],
}

rnd_search_bn = RandomizedSearchCV(DNNClassifier(random_state=42), param_distribs, n_iter=50,
                                   fit_params={"X_valid": validation_X, "y_valid": validation_y, "n_epochs": 1000},
                                   random_state=42, verbose=2)
rnd_search_bn.fit(train_X, train_y)

C:\Users\daniel\Anaconda5.0.1-Python3.6\lib\site-packages\sklearn\model_selection\_search.py:584: DeprecationWarning: "fit_params" as a constructor argument was deprecated in version 0.19 and will be removed in version 0.21. Pass fit parameters to the "fit" method instead.
  '"fit" method instead.', DeprecationWarning)


Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>, n_neurons=100, n_hidden_layers=10, learning_rate=0.01, batch_size=500, batch_norm_momentum=0.98, activation=<function leaky_relu.<locals>.parameterized_leaky_relu at 0x000000DF1CAFE620> 
0	Validation loss: 1.578798	Best loss: 1.578798	Accuracy: 90.30%
1	Validation loss: 0.207806	Best loss: 0.207806	Accuracy: 97.07%
2	Validation loss: 0.074241	Best loss: 0.074241	Accuracy: 98.55%
3	Validation loss: 0.071935	Best loss: 0.071935	Accuracy: 98.44%
4	Validation loss: 0.126832	Best loss: 0.071935	Accuracy: 97.50%
5	Validation loss: 0.087316	Best loss: 0.071935	Accuracy: 98.40%
6	Validation loss: 0.065595	Best loss: 0.065595	Accuracy: 98.63%
7	Validation loss: 0.066045	Best loss: 0.065595	Accuracy: 98.40%
8	Validation loss: 0.059083	Best loss: 0.059083	Accuracy: 98.75%
9	Validation loss: 0.073120	Best loss: 0.059083	Accuracy: 98.48%
10	Validation loss: 0.0

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   56.4s remaining:    0.0s


0	Validation loss: 0.933634	Best loss: 0.933634	Accuracy: 91.56%
1	Validation loss: 0.125035	Best loss: 0.125035	Accuracy: 97.93%
2	Validation loss: 0.111475	Best loss: 0.111475	Accuracy: 97.65%
3	Validation loss: 0.064126	Best loss: 0.064126	Accuracy: 98.59%
4	Validation loss: 0.067551	Best loss: 0.064126	Accuracy: 98.40%
5	Validation loss: 0.087713	Best loss: 0.064126	Accuracy: 98.28%
6	Validation loss: 0.057930	Best loss: 0.057930	Accuracy: 98.55%
7	Validation loss: 0.064094	Best loss: 0.057930	Accuracy: 98.36%
8	Validation loss: 0.103878	Best loss: 0.057930	Accuracy: 97.69%
9	Validation loss: 0.071202	Best loss: 0.057930	Accuracy: 98.59%
10	Validation loss: 0.073727	Best loss: 0.057930	Accuracy: 98.36%
11	Validation loss: 0.052966	Best loss: 0.052966	Accuracy: 98.91%
12	Validation loss: 0.059824	Best loss: 0.052966	Accuracy: 98.55%
13	Validation loss: 0.065658	Best loss: 0.052966	Accuracy: 98.48%
14	Validation loss: 0.056348	Best loss: 0.052966	Accuracy: 98.59%
15	Validation loss: 

25	Validation loss: 0.146640	Best loss: 0.146082	Accuracy: 97.07%
26	Validation loss: 0.107888	Best loss: 0.107888	Accuracy: 97.73%
27	Validation loss: 0.110255	Best loss: 0.107888	Accuracy: 97.85%
28	Validation loss: 0.085265	Best loss: 0.085265	Accuracy: 98.08%
29	Validation loss: 0.084715	Best loss: 0.084715	Accuracy: 98.16%
30	Validation loss: 0.070237	Best loss: 0.070237	Accuracy: 98.51%
31	Validation loss: 0.061499	Best loss: 0.061499	Accuracy: 98.67%
32	Validation loss: 0.109083	Best loss: 0.061499	Accuracy: 97.85%
33	Validation loss: 0.058080	Best loss: 0.058080	Accuracy: 98.98%
34	Validation loss: 0.054362	Best loss: 0.054362	Accuracy: 98.75%
35	Validation loss: 0.048362	Best loss: 0.048362	Accuracy: 98.91%
36	Validation loss: 0.049432	Best loss: 0.048362	Accuracy: 98.94%
37	Validation loss: 0.058090	Best loss: 0.048362	Accuracy: 98.79%
38	Validation loss: 0.064479	Best loss: 0.048362	Accuracy: 98.79%
39	Validation loss: 0.040292	Best loss: 0.040292	Accuracy: 99.06%
40	Validat

...

19	Validation loss: 0.026000	Best loss: 0.024788	Accuracy: 99.37%
20	Validation loss: 0.031023	Best loss: 0.024788	Accuracy: 99.30%
21	Validation loss: 0.029188	Best loss: 0.024788	Accuracy: 99.18%
22	Validation loss: 0.030296	Best loss: 0.024788	Accuracy: 99.34%
23	Validation loss: 0.024126	Best loss: 0.024126	Accuracy: 99.41%
24	Validation loss: 0.037327	Best loss: 0.024126	Accuracy: 99.14%
25	Validation loss: 0.025286	Best loss: 0.024126	Accuracy: 99.41%
26	Validation loss: 0.026134	Best loss: 0.024126	Accuracy: 99.37%
27	Validation loss: 0.028779	Best loss: 0.024126	Accuracy: 99.18%
28	Validation loss: 0.023677	Best loss: 0.023677	Accuracy: 99.37%
29	Validation loss: 0.024694	Best loss: 0.023677	Accuracy: 99.49%
30	Validation loss: 0.028489	Best loss: 0.023677	Accuracy: 99.34%
31	Validation loss: 0.027451	Best loss: 0.023677	Accuracy: 99.41%
32	Validation loss: 0.028312	Best loss: 0.023677	Accuracy: 99.37%
33	Validation loss: 0.032947	Best loss: 0.023677	Accuracy: 99.10%
34	Validat

8	Validation loss: 0.082295	Best loss: 0.052860	Accuracy: 98.36%
9	Validation loss: 0.062085	Best loss: 0.052860	Accuracy: 98.44%
10	Validation loss: 0.095097	Best loss: 0.052860	Accuracy: 97.50%
11	Validation loss: 0.055434	Best loss: 0.052860	Accuracy: 98.51%
12	Validation loss: 0.066142	Best loss: 0.052860	Accuracy: 98.40%
13	Validation loss: 0.068233	Best loss: 0.052860	Accuracy: 98.55%
14	Validation loss: 0.061534	Best loss: 0.052860	Accuracy: 98.48%
15	Validation loss: 0.075013	Best loss: 0.052860	Accuracy: 98.08%
16	Validation loss: 0.052796	Best loss: 0.052796	Accuracy: 98.71%
17	Validation loss: 0.063373	Best loss: 0.052796	Accuracy: 98.63%
18	Validation loss: 0.073677	Best loss: 0.052796	Accuracy: 98.67%
19	Validation loss: 0.057385	Best loss: 0.052796	Accuracy: 98.71%
20	Validation loss: 0.066551	Best loss: 0.052796	Accuracy: 98.59%
21	Validation loss: 0.071930	Best loss: 0.052796	Accuracy: 98.91%
22	Validation loss: 0.073456	Best loss: 0.052796	Accuracy: 98.83%
23	Validatio

[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 258.3min finished


0	Validation loss: 0.061790	Best loss: 0.061790	Accuracy: 98.40%
1	Validation loss: 0.045255	Best loss: 0.045255	Accuracy: 98.63%
2	Validation loss: 0.036093	Best loss: 0.036093	Accuracy: 99.10%
3	Validation loss: 0.032265	Best loss: 0.032265	Accuracy: 98.83%
4	Validation loss: 0.032538	Best loss: 0.032265	Accuracy: 98.94%
5	Validation loss: 0.034075	Best loss: 0.032265	Accuracy: 98.94%
6	Validation loss: 0.030170	Best loss: 0.030170	Accuracy: 98.98%
7	Validation loss: 0.030420	Best loss: 0.030170	Accuracy: 99.10%
8	Validation loss: 0.033145	Best loss: 0.030170	Accuracy: 98.94%
9	Validation loss: 0.030037	Best loss: 0.030037	Accuracy: 99.14%
10	Validation loss: 0.027832	Best loss: 0.027832	Accuracy: 99.22%
11	Validation loss: 0.029120	Best loss: 0.027832	Accuracy: 99.14%
12	Validation loss: 0.030427	Best loss: 0.027832	Accuracy: 98.91%
13	Validation loss: 0.026318	Best loss: 0.026318	Accuracy: 99.18%
14	Validation loss: 0.018783	Best loss: 0.018783	Accuracy: 99.37%
15	Validation loss: 

RandomizedSearchCV(cv=None, error_score='raise',
          estimator=DNNClassifier(activation=<function elu at 0x000000DF7F9CD0D0>,
       batch_norm_momentum=None, batch_size=20, dropout_rate=None,
       initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x000000DF13926A20>,
       learning_rate=0.01, n_hidden_layers=5, n_neurons=100,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42),
          fit_params={'X_valid': array([[0., 0., ..., 0., 0.],
       [0., 0., ..., 0., 0.],
       ...,
       [0., 0., ..., 0., 0.],
       [0., 0., ..., 0., 0.]], dtype=float32), 'y_valid': array([0, 4, ..., 1, 2], dtype=uint8), 'n_epochs': 1000},
          iid=True, n_iter=50, n_jobs=1,
          param_distributions={'n_neurons': [10, 30, 50, 70, 90, 100, 120, 140, 160], 'batch_size': [10, 50, 100, 500], 'learning_rate': [0.01, 0.02, 0.05, 0.1], 'activation': [<function relu at 0x000000DF7F9EA400>, <function elu at 0x000000DF7F

In [40]:
rnd_search_bn.best_estimator_.save("./8d.ckpt")